In [2]:
using DataFrames, Distributions, DataFramesMeta

In [3]:
function gen_data(
    ; 
    n = 500000, 
    mu = [0, 2], 
    sigma = 1, 
    pZ1 = 0.8)
    
    data = DataFrame(
        Obs_ID = 1:n,
        Z = (rand(Uniform(0,1), n) .> pZ1) .+ 1
        )

    @transform!(data, :Y = rand(Normal(0, sigma), n) + mu[:Z])    
    @select!(data, :Obs_ID, :Y)
    @transform!(data, :p_Y_given_Z_1 = pdf.(Normal(mu[1], sigma), :Y))
    @transform!(data, :p_Y_given_Z_2 = pdf.(Normal(mu[2], sigma), :Y))

    return data
  end

gen_data (generic function with 1 method)

In [4]:
data = gen_data(n = 500000, pZ1 = 0.8);
first(data, 6)

,Obs_ID,Y,p_Y_given_Z_1,p_Y_given_Z_2
,Int64,Float64,Float64,Float64
1,1,0.00466876,0.398938,0.0544969
2,2,1.84976,0.0720974,0.394465
3,3,-0.583821,0.336431,0.0141647
4,4,0.676837,0.317273,0.166241
5,5,2.3074,0.0278482,0.380532
6,6,0.0370222,0.398669,0.0581006


In [5]:
function fit_model!(
    data; 
    pi_hat_0 = 0.5, 
    tolerance = 0.00001,
    max_iterations = 1000,
    verbose = false
    )

    # pre-allocate a table of results by iteration:
    progress = DataFrame(
        Iteration = 0:max_iterations, 
        pi_hat = Vector{Float64}(undef, max_iterations+1), 
        loglik = Vector{Float64}(undef, max_iterations+1), 
        diff_loglik = Vector{Float64}(undef, max_iterations+1)
        )

    # initial E step, to perform needed calculations for initial likelihood:
    E_step!(data, pi_hat_0) 
    ll = loglik(data)
    progress[1, :] = (0, pi_hat_0, ll, NaN)
    
    last_iter = 0
    for i in 1:max_iterations

        # M step: re-estimate parameters
        pi_hat = M_step(data)

        # E step: re-compute distribution of missing variables, using parameters
        E_step!(data, pi_hat)
        
        # Assess convergence

        ## save the previous log-likelihood so we can test for convergence
        ll_old = ll
        
        ## here's the new log-likelihood
        ll = loglik(data)

        ll_diff = ll - ll_old
        
        progress[i+1,:] = (i, pi_hat, ll, ll_diff)
        
        if(verbose)
            print(progress[i+1,:])
        end
        
        if ll_diff < tolerance
            last_iter = i
            break
        end
    end
    
    return progress[1:(last_iter + 1), :]
end

fit_model! (generic function with 1 method)

In [6]:
function E_step!(data, pi_hat)
    @transform!(data, :pY_Z1 = :p_Y_given_Z_1 .* pi_hat)
    @transform!(data, :pY_Z2 = :p_Y_given_Z_2 .* (1- pi_hat))
    @transform!(data, :pY = :pY_Z1 + :pY_Z2)
    @transform!(data, :pZ1_given_Y = :pY_Z1 ./ :pY)
end

E_step! (generic function with 1 method)

In [7]:
function M_step(data)
    data[:, :pZ1_given_Y] |> mean
end

M_step (generic function with 1 method)

In [8]:
function loglik(data)
    data[:, :pY] .|> log |> sum
end

loglik (generic function with 1 method)

In [9]:
@time progress = fit_model!(data, tolerance = 0.00001);


  1.442003 seconds (6.98 M allocations: 881.487 MiB, 10.04% gc time, 82.27% compilation time)


The first run requires compilation; on subsequent calls it will be faster:

In [10]:
@time progress = fit_model!(data, tolerance = 0.00001);

  0.165364 seconds (13.08 k allocations: 504.204 MiB, 24.07% gc time)


Note that the memory allocation is also lower this time, because the first run expands the `DataFrame` `data` with additional columns which are reused in the second run:

In [12]:
first(data, 6)

,Obs_ID,Y,p_Y_given_Z_1,p_Y_given_Z_2,pY_Z1,pY_Z2,pY
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,0.00466876,0.398938,0.0544969,0.319211,0.0108911,0.330102
2,2,1.84976,0.0720974,0.394465,0.0576889,0.078833,0.136522
3,3,-0.583821,0.336431,0.0141647,0.269196,0.00283078,0.272027
4,4,0.676837,0.317273,0.166241,0.253867,0.0332229,0.287089
5,5,2.3074,0.0278482,0.380532,0.0222828,0.0760485,0.0983313
6,6,0.0370222,0.398669,0.0581006,0.318996,0.0116113,0.330607


Here are the iterative steps and final estimate:

In [ ]:
progress